# <center>Big Data &ndash; Exercise 1</center>
## <center>Fall 2025 &ndash; Week 1 &ndash; ETH Zurich</center>

### Aims
- **After this exercise:** 
    - Understand the SQL language and its common query patterns.
    - Understand the 'table' data shape, normalization, and when they can (and should) be used.
    - be able to query data in tables with the SQL language.
- **Later in the semester:** 
    - Relate these language features and query patterns relative to other data shapes, technologies, and the languages designed to query them.
    - Understand when tables are not the appropriate shape for your data and when you can (and should) throw normalization away!

### Prerequisites
In this exercise, you will brush-up the fundamental concepts of relational databases and SQL. If you haven't taken an introductory databases course (or want to refresh your knowledge) we recommend you to read the following:

Garcia-Molina, Ullman, Widom: Database Systems: The Complete Book. Pearson, 2. Edition, 2008. (Chapters 1, 2, 3, and 6) [Available in the ETH Library] [[Online]](https://ebookcentral.proquest.com/lib/ethz/detail.action?pq-origsite=primo&docID=5832965) [[Selected solutions]](http://infolab.stanford.edu/~ullman/dscbsols/sols.html).

Or have a look at the recordings from Information Systems for Engineers - ETH Zurich, available on [[YouTube]](https://www.youtube.com/c/GhislainFournysLectures).

### Database Set-up
We will be once again working in the ExamMagicBox (you can find it in the following [[link]](https://polybox.ethz.ch/index.php/s/wa57XqDKkxRMb0q) if you have not downloaded it yet): please drag this Notebook in the folder. Just like last week, activate the docker container for the exercise sheet with `docker compose up`; please wait for the message `PostgreSQL init process complete; ready for start up` in the docker logs before proceeding! Alternatively you can start the Docker with `docker compose up -d` and wait for the command to execute: please note that you are creating the containers in the background this way. You can then type `docker compose down` when you are done.

As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

In [1]:
%reload_ext sql
%sql postgresql://postgres:example@db

Connecting to 'postgresql://postgres:***@db'

In [2]:
%%sql
SELECT version();

Running query in 'postgresql://postgres:***@db'

1 rows affected.

version
"PostgreSQL 17.5 (Debian 17.5-1.pgdg130+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit"


### Origin of the data
You can find more information on the dataset in the following links
- [Discogs](https://www.discogs.com/)
- [Discogs XML data dumps](http://data.discogs.com/)

If you do not want to use Docker or it does not work you can download the dataset from this [link](https://cloud.inf.ethz.ch/s/DtjCHTLRHT39BRN/download/discogs.dump.xz), see `postgres-init.sh` to see how to import it)

## Exercise 1: Explore the dataset
We want to first understand the dataset a bit better. You will find some queries below to help you explore the schema.

### List tables
The following query retrieves a list of tables in the database from a system table describing the current database.

In [3]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

Running query in 'postgresql://postgres:***@db'

22 rows affected.

table_name
companies
jobs
employees02
badges
comments
inventory
posthistory
postlinks
posts
tags


### List attributes/columns
The following query retrieves a list of columns from the tables in the database.

In [4]:
%%sql 
SELECT table_name, column_name, data_type, is_nullable, ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public' AND table_name IN ('artists', 'released_by', 'releases', 'tracks')
AND table_name NOT LIKE 'pg_%'
ORDER BY table_name, ordinal_position;

Running query in 'postgresql://postgres:***@db'

17 rows affected.

table_name,column_name,data_type,is_nullable,ordinal_position
artists,artist_id,integer,NO,1
artists,name,character varying,YES,2
artists,realname,text,YES,3
artists,profile,text,YES,4
artists,url,text,YES,5
released_by,release_id,integer,NO,1
released_by,artist_id,integer,NO,2
releases,release_id,integer,NO,1
releases,released,date,NO,2
releases,title,text,NO,3


### Have a look at the datasets
The following simple query gives the first 5 rows of the `artists` dataset

In [5]:
%%sql
SELECT * FROM artists LIMIT 5;

Running query in 'postgresql://postgres:***@db'

5 rows affected.

artist_id,name,realname,profile,url
1,The Persuader,Jesper Dahlbäck,None,None
2,Mr. James Barth & A.D.,Cari Lekebusch & Alexi Delano,None,None
3,Josh Wink,Joshua Winkelman,"After forming [l=Ovum Recordings] as an independent label in October 1994 with former partner [a=King Britt], Josh recorded the cult classic 'Liquid Summer'. He went on to release singles for a wide variety of revered European labels ranging from Belgium's [l=R & S Records] to England's [l=XL Recordings]. In 1995, Wink became one of the first DJ-producers to translate his hard work into mainstream success when he unleashed a string of classics including 'Don't Laugh'¸ 'I'm Ready' and 'Higher State of Consciousness' that topped charts worldwide. More recently he has had massive club hits such as 'How's Your Evening So Far' and 'Superfreak' but he has also gained a lot of attention trough his remixes for [a=FC Kahuna], [a=Paul Oakenfold], [a=Ladytron], [a=Clint Mansell], [a=Sting] and [a=Depeche Mode], among others.",http://www.joshwink.com/
4,Johannes Heil,Johannes Heil,"Electronic music producer, musician and live performer, born 3 February 1978 near the town of Bad Nauheim, Germany. Founder of [l=JH] and [l=Metatron Recordings].",http://johannes-heil.com/
5,Heiko Laux,Heiko Laux,German DJ and producer based in Berlin. He is the founder of [l=Kanzleramt].,http://www.heiko-laux.com


Naturally we could write similar queries to better understand each of the other tables.

#### With what you now know about the datasets, try to answer the following questions

1. Which concepts are modelled in the dataset and how do they relate to each other? <b>Hint</b>: how do the tables connect logically?
2. Why do you think this shape (table) was chosen for the data and why not the other shapes?
3. In which normal forms are the corresponding relations?
4. How can we denormalise the data to make some queries more efficient? <b>Hint</b>: have a look at the queries in the next session of the exercises to see if adding some columns to some tables could reduce the need to `JOIN`.
5. What potential problems could result from adding redundancy?

## Exercise 2: SQL warm-up
Now that we familiarised ourself with the tables and relationship, we will begin with several SQL queries to ease us back into the language.

<b>Practical tips:</b>
- You might want to begin by retrieving a few rows from each of the database tables to get a sense of what is stored. 
- When testing your queries, it is good practice to add a "LIMIT" clause to avoid inadvertedly retrieving hundreds of rows.

The following is an example query that contains some common SQL expressions. A complete list can be found at: https://www.postgresql.org/docs/current/sql-select.html

In [6]:
%%sql
SELECT DISTINCT
    a.name AS column1,
    COUNT(t.track_id) AS column2,
    AVG(t.duration) AS column3
FROM
    artists a
    JOIN released_by rb USING(artist_id)
    JOIN releases r USING(release_id)
    JOIN tracks t USING(release_id)
WHERE
    t.duration > 123
    AND t.title != 'My Query'
    AND r.country = 'Switzerland'
GROUP BY
    a.artist_id, a.name
HAVING
    COUNT(t.track_id) > 0
ORDER BY
    column2 DESC,
    column3 DESC
LIMIT 5;

Running query in 'postgresql://postgres:***@db'

5 rows affected.

column1,column2,column3
Various Artists,2814,349.5806680881307747
DJ Snowman,548,287.5620437956204380
DJ Noise,504,401.8650793650793651
Dave 202,425,326.8800000000000000
DJ Nonsdrome,242,306.3842975206611570


The following is a visual representation of the database schema for quick reference.

<img src="https://polybox.ethz.ch/index.php/s/8CqNffQrR0EDbuC/download" width=800/>

#### 1. Retrieve all releases that were released after January 1, 2017.

In [7]:
%%sql
SELECT *
FROM releases a
WHERE released > '2017-01-01'
LIMIT 5

Running query in 'postgresql://postgres:***@db'

5 rows affected.

release_id,released,title,country,genre
100000,2018-12-14,"Kizomba Mix, Vol. 2 [2018] 2 CDs",Portugal,Electronic
130357,2018-01-01,Shed01,UK,Electronic
138522,2018-01-01,Jetlag Disco,Germany,Electronic
264023,2018-01-01,The Bad Behaviour E.P,UK,Electronic
266607,2018-01-01,Volume 1,UK,Electronic


#### 2. Find all tracks with a duration longer than 7 hours. Assume the 'duration' column in the 'tracks' table is in seconds.

In [8]:
%%sql
SELECT * 
FROM tracks
WHERE duration > (60*60)*7

Running query in 'postgresql://postgres:***@db'

3 rows affected.

release_id,position,title,duration,track_id
47796,9,Rapper's Relight,25579,256970
47796,11,Dialectical Transformation III Peace In Rwanda Mix,27196,256972
478281,4,Live 1996.12.30.,31934,2526159


#### 3. Retrieve the titles of 5 releases along with the names of the artists who released them.

In [9]:
%%sql
SELECT r.title, a.name
FROM releases r 
    JOIN released_by rb USING (release_id)
    JOIN artists a USING (artist_id)
LIMIT 5

Running query in 'postgresql://postgres:***@db'

5 rows affected.

title,name
Stockholm,The Persuader
Knockin' Boots Vol 2 Of 2,Mr. James Barth & A.D.
Profound Sounds Vol. 1,Josh Wink
Flowerhead,DATacide
Knockin' Boots (Vol 1 Of 2),Mr. James Barth & A.D.


#### 4. List each genre and the number of releases in that genre.

In [10]:
%%sql
SELECT r.genre, COUNT(*) as numer_of_releases 
FROM releases r 
GROUP BY r.genre 
LIMIT 5

Running query in 'postgresql://postgres:***@db'

5 rows affected.

genre,numer_of_releases
Blues,48
Brass & Military,4
Children's,6
Classical,257
Electronic,183766


#### 5. Identify the top 5 artists who have the most releases.

In [11]:
%%sql
SELECT *
FROM artists
JOIN(
    SELECT artist_id, COUNT(release_id) as nor
    FROM released_by
    JOIN releases USING (release_id)
    GROUP BY artist_id
) rc USING (artist_id)
ORDER BY rc.nor DESC
LIMIT 5; 

Running query in 'postgresql://postgres:***@db'

5 rows affected.

artist_id,name,realname,profile,url,nor
194,Various Artists,None,Placeholder artist for Various,None,46123
8760,Madonna,Madonna Louise Veronica Ciccone,"American singer, entertainer, songwriter, actress, director and businesswoman. Madonna achieved popularity by pushing the boundaries of lyrical content in mainstream popular music and imagery in her music videos. Referred to as the ""Queen of Pop"", Madonna is often cited as an influence by other artists. Born August 16, 1958 in Bay City, Michigan, Madonna moved to New York City to pursue a career in modern dance. After performing in the music groups [a=Breakfast Club] and [url=http://www.discogs.com/artist/Emmy+(7)]Emmy[/url], she signed with Sire in 1982 and released her debut album, [i][m=5319][/i], the following year. She followed it with a series of commercially successful albums, has sold more than 300 million records worldwide and is recognized as the best-selling female recording artist of all time by Guinness World Records. Madonna was inducted into the Rock and Roll Hall of Fame in her first year of eligibility. She was ranked at number one on VH1's list of 100 Greatest Women in Music, and at number two on Billboard's list of Greatest Hot 100 Artists of All Time (behind only The Beatles), the latter making her the most successful solo artist in the history of American singles chart.",http://www.madonna.com,617
7552,Pet Shop Boys,"Neil Tennant, Chris Lowe","Pet Shop Boys are an English musical duo, formed in London in 1981 and consisting of Neil Tennant (main vocals, keyboards, occasional guitar) and Chris Lowe (keyboards, occasional vocals). They are listed as the most successful duo in UK music history by The Guinness Book of Records. In the early 90s they ran [l=Spaghetti Recordings] and more recently, [l=Olde English] and [l=Lucky Kunst] labels. In March 2013, the duo officially left [l=Parlophone] after 28 years and entered into a new arrangement with [l=Kobalt Label Services] for their 12th studio album which would be the band's first release on their own music imprint [l=x2 (2)].",http://www.petshopboys.co.uk (Official Site),600
4118,Faithless,None,"Faithless officially split up in April 2011. Some of the original members (Sister Bliss, Maxi Jazz and percussionist Sudha Kheterpal) continue to perform under the name Faithless Sound System. Faithless were [a=Maxwell Frazer] AKA [a=Maxi Jazz] (Conscious Rap) [a=Ayalah Bentovim] AKA [a=Sister Bliss] (Music) Roland Armstrong AKA [a=Rollo] (Producer, Music). Many of the songs are sung by [a=Pauline Taylor]. [a=Jamie Catto] was originally part of the group but left after the release of the second album ""Sunday 8PM"". [a=Zoë Johnston] joined for the album ""Outrospective"". Their first album, 1996's ""Reverence"", was an underground sensation but did not quite break the mainstream. 1998's ""Sunday 8AM"" changed all of that, the single ""God Is a DJ"" became a big club anthem while the album enjoyed bigger sales than anything they had previously released. Critics may have been harsh, but the group kept moving forward with all members also pursuing solo careers on the side. When playing live they were joined by guitarist [a=Dave Randall (2)] and bassist, [a=Aubrey Nunn].",http://www.faithless.co.uk,336
15885,Michael Jackson,Michael Joseph Jackson,"American singer, dancer, entertainer, songwriter, producer and recording artist. Born: 29 August 1958 in Gary, Indiana, USA. Died: 25 June 2009 in Los Angeles, California, USA (aged 50). Known affectionately as the ""King Of Pop"", Jackson was a singer, dancer, musician, music producer, writer, entertainer, singer-songwriter, choreographer, record producer, recording artist, poet, arranger, businessman, philanthropist, actor, voice artist, and comedian. Jackson began his career as the youngest member of [a=The Jackson 5] and started his solo recording career in 1971. Brother of recording artists [a=Jackie Jackson], [a=Janet Jackson], [a=Jermaine Jackson], [a=La Toya Jackson], 

#### 6. Find the artist who has the longest total duration of tracks across all their releases.

In [12]:
%%sql 
SELECT *
FROM artists a
JOIN (
    SELECT artist_id, SUM(tracks.duration) as td
    FROM released_by
    JOIN releases USING (release_id) 
    JOIN tracks USING (release_id)
    GROUP BY artist_id
) duration_sum USING (artist_id)
ORDER BY duration_sum.td DESC
LIMIT 1

Running query in 'postgresql://postgres:***@db'

1 rows affected.

artist_id,name,realname,profile,url,td
194,Various Artists,None,Placeholder artist for Various,None,227180023


#### 7. Find how many releases that have tracks with duplicate titles.

In [13]:
%%sql
SELECT COUNT(*) as number_of_artists
FROM artists
JOIN(
    SELECT artist_id, COUNT(*) as classical_count
    FROM released_by
    JOIN releases USING (release_id)
    WHERE releases.genre = 'Classical'
    GROUP BY artist_id
) cc USING (artist_id) 
JOIN(
    SELECT artist_id, COUNT(*) as jazz_count
    FROM released_by
    JOIN releases USING (release_id)
    WHERE releases.genre = 'Jazz'
    GROUP BY artist_id
) jc USING (artist_id) 
WHERE artists.name <> 'Various Artists' AND classical_count > 2 AND jazz_count > 1

Running query in 'postgresql://postgres:***@db'

1 rows affected.

number_of_artists
4


In [14]:
%%sql
SELECT *
FROM artists
JOIN(
    SELECT artist_id, COUNT(*) as rock_count
    FROM released_by
    JOIN releases USING (release_id)
    WHERE releases.genre = 'Rock'
    GROUP BY artist_id
) rc USING (artist_id) 
where artists.name <> 'Various Artists'
ORDER BY rc.rock_count DESC
LIMIT 1 OFFSET 1; 

Running query in 'postgresql://postgres:***@db'

1 rows affected.

artist_id,name,realname,profile,url,rock_count
94078,The Kinks,"Ray Davies, Dave Davies, Mick Avory, Pete Quaife","The Kinks are one of the most important bands from the ""British Invasion"" in the sixties. The band, based in Muswell Hill in London, consisted of the brothers Davies, (Ray and Dave), Mick Avory and Pete Quaife. During their existence they have played different styles of rock ('n' roll) music. Interesting were their lyrics, usually about the lower class of society. Singer Ray Davies has always had a fascination for the ordinary people. After two flops they had their first big hit in 1964: You Really Got Me. This song had a rough guitar riff, that's why some people even call it the first heavy metal song ever. They kept scoring hits after that, timeless songs like Sunny Afternoon, Waterloo Sunset, Lola and All Day And All Of The Night. Albums such as The Village Green Preservation Society (1968) and Arthur (1969) are seen as classics nowadays. Pete Quaife left The Kinks during the [m=101386] rehearsals. In the beginning of the seventies it was over with the hit singles. They started a series of rockopera's, which were not very succesful. In 1977, with the release of Sleepwalker, they went back to more normal rock. Their popularity started to rise again, especially in America. They have produced records until 1996, some with more success than others. Ray and Dave are both doing solo tours and releasing solo albums. In a June 2018 interview, Ray Davies stated that he, along with brother Dave, and drummer Avory, had reformed The Kinks for a new studio album and to potentially perform live. Inducted into Rock And Roll Hall of Fame in 1990 (Performer). Pete Quaife, who had been receiving kidney dialysis for more than ten years, died on 23 June 2010, aged 66.",http://www.thekinks.info/,134


#### 8. Retrieve the artists with the name of 'Coldplay'.

In [15]:
%%sql
SELECT *
FROM artists 
WHERE name = 'Coldplay'
LIMIT 5


Running query in 'postgresql://postgres:***@db'

1 rows affected.

artist_id,name,realname,profile,url
29735,Coldplay,None,"Coldplay is an English rock band from London, England. They've been a band since January 16, 1998 when they lost a demotape competition on XFM in London. Philip Christopher Harvey is the band's manager. [b][u]Line-up:[/u][/b] Jonny Buckland (Jonathan Mark Buckland) - Guitar Will Champion (William Champion) - Drums Guy Berryman (Guy Rupert Berryman) - Bass Chris Martin (Christopher Anthony John Martin) - Vocals",http://coldplay.com/


#### 9. List the titles of all releases by that artist in alphabetical order.
<b>Hint</b>: Ignore the fact that different relases can have the same title.

In [16]:
%%sql
SELECT DISTINCT r.title
FROM artists a
    JOIN released_by rb USING (artist_id)
    JOIN releases r USING (release_id)
WHERE a.name = 'Coldplay'
ORDER BY r.title
LIMIT 5

Running query in 'postgresql://postgres:***@db'

5 rows affected.

title
Acoustic
A Rush Of Blood To The Head
Boot Of Sound
Brothers & Sisters
Clocks


#### 10. How many tracks from 'Coldplay' have position '1'?

In [17]:
%%sql
SELECT DISTINCT COUNT(*) 
FROM artists a
    JOIN released_by rb USING (artist_id)
    JOIN releases r USING (release_id)
    JOIN tracks t USING (release_id)
WHERE a.name = 'Coldplay' AND t.position = '1'
LIMIT 5

Running query in 'postgresql://postgres:***@db'

1 rows affected.

count
32


#### 11. List the titles of all releases by Coldplay that contain less than 2 tracks.

In [18]:
%%sql
SELECT DISTINCT r.title
FROM artists a
JOIN released_by rb USING (artist_id)
JOIN releases r USING (release_id)
JOIN tracks t USING (release_id)
WHERE a.name = 'Coldplay'
GROUP BY t.release_id, r.title
HAVING COUNT(*) < 2
LIMIT 5;

Running query in 'postgresql://postgres:***@db'

5 rows affected.

title
Boot Of Sound
Clocks
Clocks (Cosmos Rmx)
Clocks (Dean Coleman Remix)
Clocks (Remix)


#### 12. What is the average track duration?

In [19]:
%%sql
SELECT AVG(t.duration) 
    FROM tracks t

Running query in 'postgresql://postgres:***@db'

1 rows affected.

avg
325.0749298696787647


#### What is the average track duration of Coldplay songs?

In [20]:
%%sql
SELECT AVG(t.duration)
FROM artists a
JOIN released_by rb USING (artist_id)
JOIN releases r USING (release_id)
JOIN tracks t USING (release_id)
WHERE a.name = 'Coldplay'

Running query in 'postgresql://postgres:***@db'

1 rows affected.

avg
292.0936454849498328


#### 13. How many artists have released tracks longer than twice the average?

In [21]:
%%sql
SELECT COUNT(DISTINCT rb.artist_id)
FROM tracks t
JOIN releases r USING (release_id)
JOIN released_by rb USING (release_id)
WHERE  2*(SELECT AVG(t.duration) FROM tracks t) < t.duration 
LIMIT 5

Running query in 'postgresql://postgres:***@db'

1 rows affected.

count
6386


## Exercise 3: more SQL
We will now see more complex SQL queries.

<b>Practical tips:</b>

When writing complex queries, you might want to split them into smaller parts by using <b>Common Table Expressions</b> (CTEs). A CTE is a named temporary result set that you can reference within statements (SELECT, INSERT, UPDATE, ... ). You can find more about CTEs at: https://www.postgresql.org/docs/current/queries-with.html

The following is an example of a query using two CTEs:

In [22]:
%%sql
WITH countries AS (
    SELECT DISTINCT country FROM releases
),
genres AS (
    SELECT DISTINCT genre FROM releases
)
SELECT c.country as column1, g.genre as column2
FROM countries c, genres g
LIMIT 5;

Running query in 'postgresql://postgres:***@db'

5 rows affected.

column1,column2
Argentina,Blues
Argentina,Brass & Military
Argentina,Children's
Argentina,Classical
Argentina,Electronic


In some exercises, you might also want to use <b>subqueries</b>. A subquery is a nested query, usually with the purpose of retrieving data that will be used in in the outer query. For instance, subqueries can appear in WHERE, FROM and SELECT clauses.

The following is an example of a query than includes a subquery:

In [23]:
%%sql
SELECT release_id as column1 FROM (
    SELECT release_id, title, COUNT(*) FROM tracks
    GROUP BY release_id, title
    HAVING COUNT(*) > 1
) sub
LIMIT 5;

Running query in 'postgresql://postgres:***@db'

5 rows affected.

column1
110
466
695
695
710


#### 1. What is the title of the album from 'Coldplay' with the most amount of tracks?

In [24]:
%%sql

-- RETURNS NUMBER OF TRACKS PER RELEASE
WITH counter AS (SELECT r.release_id, COUNT(r.release_id) AS cnt
FROM releases r 
    JOIN tracks t USING (release_id)
GROUP BY r.release_id
)
    
SELECT r.title, c.cnt
FROM artists a
    JOIN released_by rb USING (artist_id)
    JOIN releases r USING (release_id)
    JOIN counter c USING (release_id)
WHERE a.name = 'Coldplay'
GROUP BY r.release_id, r.title, c.cnt
ORDER BY c.cnt DESC
LIMIT 1

Running query in 'postgresql://postgres:***@db'

1 rows affected.

title,cnt
X&Y (Special Dutch Edition),19


#### 2. What is the name of the first artist in alphabetical order with releases in the most genres. Please make sure to exclude "Various Artists".

In [25]:
%%sql

SELECT a.name, COUNT(DISTINCT r.genre) as genre_c
FROM artists a
    JOIN released_by rb USING (artist_id)
    JOIN releases r USING (release_id)
    WHERE a.name <> 'Various Artists'
    GROUP BY a.name
    ORDER BY genre_c DESC, a.name ASC
LIMIT 1

    

Running query in 'postgresql://postgres:***@db'

1 rows affected.

name,genre_c
Diana Ross,7


#### 3. In what year did they (the artist from the previous question) release their first album?

In [26]:
%%sql
-- EXTRACT (YEAR FROM <date>) takes only the YEAR
SELECT EXTRACT(YEAR FROM MIN(r.released)) as first_release
FROM artists a
    JOIN released_by rb USING (artist_id)
    JOIN releases r USING (release_id)
    WHERE a.name = 'Diana Ross'


Running query in 'postgresql://postgres:***@db'

1 rows affected.

first_release
1967


#### 4. How many artists have released an album with total track duration above twice the average total track duration?

<b>Hint</b>: this is not the same as exercise 2.13 since we are lookong at the <b>total</b> track duration of the album.

In [27]:
%%sql
-- total duration of single releases (albums) 
WITH total_duration AS (
    SELECT r.release_id, SUM(t.duration) AS total
    FROM releases r
        JOIN tracks t USING (release_id)
    GROUP BY r.release_id
)
SELECT COUNT(DISTINCT a.artist_id)
FROM artists a 
    JOIN released_by rb USING (artist_id)
    JOIN total_duration td USING (release_id)
    WHERE td.total > 2*(SELECT AVG(td.total) 
        FROM total_duration td
    )


Running query in 'postgresql://postgres:***@db'

1 rows affected.

count
3388


#### 5. How many artists have both a release with a track longer than twice the average and one with total duration longer than twice the average?

<b>Hint</b>: you can use `INTERSECT` or `EXISTS` to write your query.

In [28]:
%%sql
WITH total_duration AS (
    SELECT release_id, SUM(duration) AS duration
    FROM tracks
    GROUP BY release_id
),
average AS (
    SELECT AVG(duration) AS avg_duration
    FROM total_duration
),
average_duration AS (
    SELECT AVG(CAST(duration AS FLOAT)) AS avg_track_duration
    FROM tracks
),
artist_intersection AS (
    SELECT artist_id
    FROM artists a
    JOIN released_by rb USING(artist_id)
    JOIN releases r USING(release_id)
    JOIN total_duration td USING(release_id)
    WHERE td.duration > 2*(SELECT avg_duration FROM average)
    INTERSECT
    SELECT artist_id
    FROM artists a
    JOIN released_by rb USING(artist_id)
    JOIN releases r USING(release_id)
    JOIN tracks t USING(release_id)
    WHERE t.duration > 2*(SELECT avg_track_duration FROM average_duration)
)

SELECT COUNT(*)
FROM artist_intersection;

Running query in 'postgresql://postgres:***@db'

1 rows affected.

count
1870


#### 6. Show the artists have more than 200 releases in total but have no releases with the genre 'Pop' in reversed alphabetical order.

In [29]:
%%sql 
-- Select those artists that do not have pop as a genre
WITH no_pop AS (
    SELECT r.release_id
    FROM releases r
    WHERE r.genre <> 'Pop'
),
-- COUNTS how many releases each artists has
above_200 AS (
    SELECT rb.artist_id, COUNT(rb.release_id) AS cnt
    FROM artists a 
        JOIN released_by rb USING (artist_id)
        JOIN releases r USING (release_id)
    GROUP BY rb.artist_id
),
-- USE INTERSECTION TO FIND THE ONES that have both 
artist_intersect AS (
    SELECT ab.artist_id
    FROM above_200 ab
    WHERE ab.cnt > 200
    INTERSECT
    SELECT a.artist_id
    FROM no_pop np
        JOIN released_by r USING (release_id)
        JOIN artists a USING (artist_id)
)

SELECT a.name
FROM artist_intersect ai
JOIN artists a USING (artist_id)
ORDER BY a.name DESC


Running query in 'postgresql://postgres:***@db'

15 rows affected.

name
Various Artists
Underworld
The Shamen
The Art Of Noise
Technotronic
Tangerine Dream
Pet Shop Boys
Orbital
Michael Jackson
Madonna


## Exercise 4: Discuss query patterns and language features of SQL
1. What patterns did you use in many of the queries above? 

2. What is the usual pattern of an SQL query? Which operations happen pre-grouping and which ones post-grouping?

3. What makes SQL a declarative language and what advantages does that have?

4. What makes SQL a functional language and what advantages does that have?

5. How would the denormalization we talked about previously simplify the queries?

## Exercise 5: Limits of SQL (optional)
Explain what the following query does.
<b>Hints</b>: The query treats the data as if it was in graph shape.

In [30]:
%%sql
WITH RECURSIVE
    X AS (SELECT 3 AS Value),
    artist_releases AS (
        SELECT artists.artist_id, artists.name, releases.release_id, releases.title
        FROM artists, released_by, releases
        WHERE artists.artist_id = released_by.artist_id
        AND released_by.release_id = releases.release_id
    ),
    collaborations AS (
        SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
                ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
        FROM artist_releases AS ar1, artist_releases AS ar2
        WHERE ar1.release_id = ar2.release_id
        AND ar1.artist_id != ar2.artist_id
    ),
    X_hop_collaborations AS (
        SELECT * FROM collaborations  -- base case
        UNION
        SELECT c1.left_id, c1.left_name, c2.right_id, c2.right_name, c1.distance + 1 AS distance
        FROM X_hop_collaborations AS c1
        JOIN collaborations c2 ON c1.right_id = c2.left_id
        WHERE c1.distance < (SELECT * FROM X) AND c1.left_id != c2.right_id
    )
SELECT * 
FROM X_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY distance, right_name;

Running query in 'postgresql://postgres:***@db'

23 rows affected.

left_id,left_name,right_id,right_name,distance
29735,Coldplay,1654,DK,1
29735,Coldplay,392179,G Synth,1
29735,Coldplay,10916,Jan Johnston,1
29735,Coldplay,1279,Orbital,1
29735,Coldplay,10785,Angelo Badalamenti,2
29735,Coldplay,11101,Cosmic Gate,2
29735,Coldplay,7090,Freefall,2
29735,Coldplay,2604010,Jada (7),2
29735,Coldplay,18836,Kirk Hammett,2
29735,Coldplay,11749,Knuckleheadz,2


In [31]:
#### __LETS ANALYZE THE FIRST 2 CTE__

FOLLOWING CODE PAIRS EACH ARTISTS TO ITS RELEASES

In [32]:
%%sql
SELECT artists.artist_id, artists.name, releases.release_id, releases.title
        FROM artists, released_by, releases
        WHERE artists.artist_id = released_by.artist_id
        AND released_by.release_id = releases.release_id

Running query in 'postgresql://postgres:***@db'

229533 rows affected.

artist_id,name,release_id,title
22,DATacide,5,Flowerhead
2,Mr. James Barth & A.D.,6,Knockin' Boots (Vol 1 Of 2)
58,DaRand Land,23,Blessings
55,DJ Rasoul,27,Soul Searching Vol. 2
55,DJ Rasoul,31,Soul Searching Vol. 4
73,Harry The Bastard,37,"Club ""H"""
92,Fredrik Stark,42,Retrospect
12007,Robert Rich,48,Stalker
34,Miguel Migs,51,Take Me To Paradise
159,Savvas Ysatis,65,Highrise


THE FOLLOWING CODE CHECKS WHETHER A RELEASE IS AMPPED TO TWO ARTISTS

In [33]:
%%sql
WITH artist_releases AS (
    SELECT artists.artist_id, artists.name, releases.release_id, releases.title
    FROM artists, released_by, releases
    WHERE artists.artist_id = released_by.artist_id
    AND released_by.release_id = releases.release_id
),
collaborations AS (
        SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
                ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
        FROM artist_releases AS ar1, artist_releases AS ar2
        WHERE ar1.release_id = ar2.release_id
        AND ar1.artist_id != ar2.artist_id
    )
SELECT * FROM collaborations

Running query in 'postgresql://postgres:***@db'

11592 rows affected.

left_id,left_name,right_id,right_name,distance
1,The Persuader,89,Mr. James Barth,1
3,Josh Wink,2893,Woody McBride,1
3,Josh Wink,27460,The Interpreters,1
3,Josh Wink,36856,20 Fingers,1
4,Johannes Heil,5,Heiko Laux,1
4,Johannes Heil,64895,Felipe,1
4,Johannes Heil,114532,Hell,1
4,Johannes Heil,811025,The Hidden (2),1
5,Heiko Laux,4,Johannes Heil,1
5,Heiko Laux,57279,Sammy Dee,1


SO IN THE END

CREATES AN UNDIRECTED GRAPH AND FINDS THE ARTISTS THAT ARE 3 HOPS AWAY FROM A COLLABORATION POINT OF VIEW